In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crater-segmentation/Ytrain2_b.npy
/kaggle/input/crater-segmentation/Xtrain2_b.npy


In [ ]:
X = np.load('Xtrain2_b.npy')
y = np.load('Ytrain2_b.npy')

In [3]:
print(X.shape,y.shape)

(547, 2304) (547, 2304)


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ColorJitter, Resize
from torchvision.transforms.functional import InterpolationMode
from torch.cuda.amp import autocast, GradScaler
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

# Check GPU availability and set device
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

class CraterDataset(Dataset):
    def __init__(self, images, masks, processor, transform=None):
        self.images = images
        self.masks = masks
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
        
        # Apply ColorJitter augmentation if transform exists
        if self.transform and torch.rand(1) < 0.5:  # 50% chance to apply
            image_tensor = torch.from_numpy(image_rgb).permute(2, 0, 1)
            image_tensor = self.transform(image_tensor)
            image_rgb = image_tensor.permute(1, 2, 0).numpy()

        inputs = self.processor(
            images=image_rgb, 
            segmentation_maps=mask, 
            return_tensors="pt",
            do_rescale=False
        )
        return {
            'pixel_values': inputs['pixel_values'].squeeze(),
            'labels': inputs['labels'].squeeze()
        }

def calculate_validation_accuracy(model, X_test_normalized, y_test, processor):
    resize_transform = Resize(
        size=(48, 48),
        interpolation=InterpolationMode.BILINEAR,
        antialias=True
    )
    
    model.eval()
    predictions = np.zeros((len(X_test_normalized), 48, 48))
    
    with torch.no_grad():
        for i, image in enumerate(X_test_normalized):
            image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
            inputs = processor(
                images=image_rgb,
                return_tensors="pt",
                do_rescale=False
            )
            pixel_values = inputs['pixel_values'].cuda()
            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
            pred = probs[0, 1].cpu().numpy()
            
            if pred.shape != (48, 48):
                pred_tensor = torch.from_numpy(pred).unsqueeze(0)
                pred_resized = resize_transform(pred_tensor).squeeze().numpy()
                predictions[i] = pred_resized
            else:
                predictions[i] = pred
    
    y_true_flat = y_test.reshape(-1, 48, 48).flatten()
    y_pred_flat = predictions.flatten()
    y_pred_flat = (y_pred_flat > 0.5).astype(int)
    
    return balanced_accuracy_score(y_true_flat, y_pred_flat)

print("Starting data preparation...")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")

# Reshape and normalize data
X_train_reshaped = X_train.reshape(-1, 48, 48, 1)
y_train_reshaped = y_train.reshape(-1, 48, 48)
X_train_normalized = X_train_reshaped / 255.0

# Reshape and normalize test data
X_test_reshaped = X_test.reshape(-1, 48, 48, 1)
X_test_normalized = X_test_reshaped / 255.0

print("Initializing model and processor...")

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b3")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=2,
    ignore_mismatched_sizes=True
)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model = model.cuda()

# Data augmentation
color_jitter = ColorJitter(
    brightness=0.2,
    contrast=0.2,
    saturation=0.2,
    hue=0.1
)

# Create dataset and dataloader with augmentation
train_dataset = CraterDataset(
    X_train_normalized, 
    y_train_reshaped, 
    processor,
    transform=color_jitter
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Training setup
optimizer = torch.optim.AdamW([
    {'params': model.module.segformer.parameters(), 'lr': 1e-5},
    {'params': model.module.decode_head.parameters(), 'lr': 1e-4}
])

scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=5,
    T_mult=2,
    eta_min=1e-6
)

scaler = GradScaler()

# Training loop
best_balanced_accuracy = 0
patience = 100  # Changed to 20
patience_counter = 0
num_epochs = 100  # Changed to 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    batch_count = 0
    
    for batch_idx, batch in enumerate(train_dataloader):
        pixel_values = batch['pixel_values'].cuda()
        labels = batch['labels'].cuda()
        
        with autocast():
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss.mean()
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        batch_count += 1
        
        if batch_idx % 5 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}")
            print(f"Learning rates: {[group['lr'] for group in optimizer.param_groups]}")
            for i in range(torch.cuda.device_count()):
                memory_allocated = torch.cuda.memory_allocated(i) / 1e9
                print(f"GPU {i} memory allocated: {memory_allocated:.2f} GB")
    
    avg_loss = total_loss / batch_count
    
    # Calculate validation balanced accuracy
    val_balanced_accuracy = calculate_validation_accuracy(model, X_test_normalized, y_test, processor)
    
    print(f"Epoch: {epoch}")
    print(f"Average Training Loss: {avg_loss:.4f}")
    print(f"Validation Balanced Accuracy: {val_balanced_accuracy:.4f}")
    
    scheduler.step()
    
    # Early stopping and model saving based on balanced accuracy
    if val_balanced_accuracy > best_balanced_accuracy:
        best_balanced_accuracy = val_balanced_accuracy
        patience_counter = 0
        # Save best model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': avg_loss,
            'balanced_accuracy': val_balanced_accuracy,
        }, 'best_model.pt')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs - No improvement in balanced accuracy")
            break
    
    # Regular checkpoint saving
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': avg_loss,
            'balanced_accuracy': val_balanced_accuracy,
        }, f'checkpoint_epoch_{epoch+1}.pt')

CUDA available: True
Number of GPUs: 2
GPU 0: Tesla T4
GPU 1: Tesla T4
Starting data preparation...
Training set size: (437, 2304), Test set size: (110, 2304)
Initializing model and processor...


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

...

Epoch: 98
Average Training Loss: 0.0662
Validation Balanced Accuracy: 0.8185


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/tmp/ipykernel_29/458524710.py:173: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch: 99, Batch: 0, Loss: 0.0579
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 5, Loss: 0.0550
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 10, Loss: 0.0691
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 15, Loss: 0.0660
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 20, Loss: 0.0517
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 25, Loss: 0.0751
Learning rates: [8.14503363531613e-06, 7.959536998847742e-05]
GPU 0 memory allocated: 0.85 GB
GPU 1 memory allocated: 0.02 GB
Epoch: 99, Batch: 30, Loss: 0.0868
L

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 99
Average Training Loss: 0.0656
Validation Balanced Accuracy: 0.8247


In [7]:
import torch
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
import numpy as np
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode

def evaluate_model_on_test_set(model, X_test_normalized, y_test, processor):
    resize_transform = Resize(
        size=(48, 48),
        interpolation=InterpolationMode.BILINEAR,
        antialias=True
    )
    
    model.eval()
    predictions = torch.zeros((len(X_test_normalized), 48, 48)).cuda()
    y_test_tensor = torch.from_numpy(y_test.reshape(-1, 48, 48)).cuda()
    
    with torch.no_grad():
        for i, image in enumerate(X_test_normalized):
            image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
            inputs = processor(
                images=image_rgb,
                return_tensors="pt",
                do_rescale=False
            )
            pixel_values = inputs['pixel_values'].cuda()
            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
            pred = probs[0, 1]
            
            if pred.shape != (48, 48):
                pred = resize_transform(pred.unsqueeze(0)).squeeze()
            predictions[i] = pred
    
    predictions = (predictions > 0.5).float()
    y_true_flat = y_test_tensor.flatten()
    y_pred_flat = predictions.flatten()
    
    # Calculate metrics
    tp = torch.sum((y_true_flat == 1) & (y_pred_flat == 1)).float()
    tn = torch.sum((y_true_flat == 0) & (y_pred_flat == 0)).float()
    fp = torch.sum((y_true_flat == 0) & (y_pred_flat == 1)).float()
    fn = torch.sum((y_true_flat == 1) & (y_pred_flat == 0)).float()
    
    sensitivity = tp / (tp + fn + 1e-7)
    specificity = tn / (tn + fp + 1e-7)
    balanced_acc = (sensitivity + specificity) / 2
    
    return {
        'balanced_accuracy': balanced_acc.item(),
        'sensitivity': sensitivity.item(),
        'specificity': specificity.item(),
        'true_positives': tp.item(),
        'true_negatives': tn.item(),
        'false_positives': fp.item(),
        'false_negatives': fn.item()
    }

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b3")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=2,
    ignore_mismatched_sizes=True
).cuda()

# Load the best checkpoint
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])

# Evaluate the model
metrics = evaluate_model_on_test_set(model, X_test_normalized, y_test, processor)

# Print results
print("\nTest Set Metrics:")
print(f"Balanced Accuracy: {metrics['balanced_accuracy']:.4f}")
print(f"Sensitivity: {metrics['sensitivity']:.4f}")
print(f"Specificity: {metrics['specificity']:.4f}")
print("\nDetailed Counts:")
print(f"True Positives: {metrics['true_positives']}")
print(f"True Negatives: {metrics['true_negatives']}")
print(f"False Positives: {metrics['false_positives']}")
print(f"False Negatives: {metrics['false_negatives']}")

/opt/conda/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN thi

RuntimeError: Error(s) in loading state_dict for SegformerForSemanticSegmentation:
	Missing key(s) in state_dict: "segformer.encoder.patch_embeddings.0.proj.weight", "segformer.encoder.patch_embeddings.0.proj.bias", "segformer.encoder.patch_embeddings.0.layer_norm.weight", "segformer.encoder.patch_embeddings.0.layer_norm.bias", "segformer.encoder.patch_embeddings.1.proj.weight", "segformer.encoder.patch_embeddings.1.proj.bias", "segformer.encoder.patch_embeddings.1.layer_norm.weight", "segformer.encoder.patch_embeddings.1.layer_norm.bias", "segformer.encoder.patch_embeddings.2.proj.weight", "segformer.encoder.patch_embeddings.2.proj.bias", "segformer.encoder.patch_embeddings.2.layer_norm.weight", "segformer.encoder.patch_embeddings.2.layer_norm.bias", "segformer.encoder.patch_embeddings.3.proj.weight", "segformer.encoder.patch_embeddings.3.proj.bias", "segformer.encoder.patch_embeddings.3.layer_norm.weight", "segformer.encoder.patch_embeddings.3.layer_norm.bias", "segformer.encoder.blo

In [43]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from sklearn.model_selection import train_test_split

# Check GPU availability and set device
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

class CraterDataset(Dataset):
    def __init__(self, images, masks, processor, transform=None):
        self.images = images
        self.masks = masks
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
        
        if self.transform:
            image_tensor = torch.from_numpy(image_rgb).permute(2, 0, 1)
            image_tensor = self.transform(image_tensor)
            image_rgb = image_tensor.permute(1, 2, 0).numpy()

        inputs = self.processor(
            images=image_rgb, 
            segmentation_maps=mask, 
            return_tensors="pt",
            do_rescale=False
        )
        return {
            'pixel_values': inputs['pixel_values'].squeeze(),
            'labels': inputs['labels'].squeeze()
        }

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_epoch(model, train_loader, optimizer, scheduler, scaler, device, gradient_accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    for batch_idx, batch in enumerate(train_loader):
        with autocast():
            outputs = model(
                pixel_values=batch['pixel_values'].to(device),
                labels=batch['labels'].to(device)
            )
            loss = outputs.loss.mean() / gradient_accumulation_steps
        
        scaler.scale(loss).backward()
        
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        
        total_loss += loss.item() * gradient_accumulation_steps
        
        if batch_idx % 5 == 0:
            print(f"Batch: {batch_idx}, Loss: {loss.item()*gradient_accumulation_steps:.4f}")
            
    return total_loss / len(train_loader)

def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            outputs = model(
                pixel_values=batch['pixel_values'].to(device),
                labels=batch['labels'].to(device)
            )
            loss = outputs.loss.mean()
            total_loss += loss.item()
    return total_loss / len(val_loader)

print("Starting data preparation...")

# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Reshape and normalize data
X_train_reshaped = X_train.reshape(-1, 48, 48, 1)
X_val_reshaped = X_val.reshape(-1, 48, 48, 1)
y_train_reshaped = y_train.reshape(-1, 48, 48)
y_val_reshaped = y_val.reshape(-1, 48, 48)

X_train_normalized = X_train_reshaped / 255.0
X_val_normalized = X_val_reshaped / 255.0

print("Initializing model and processor...")

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b5")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b5",
    num_labels=2,
    ignore_mismatched_sizes=True
)

print(f"Model Parameters: {count_parameters(model):,}")

# Move model to GPU and enable data parallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model = model.cuda()

# Data augmentation
transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5)
])

# Create datasets
train_dataset = CraterDataset(X_train_normalized, y_train_reshaped, processor, transform=transform)
val_dataset = CraterDataset(X_val_normalized, y_val_reshaped, processor, transform=None)

# Create dataloaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# Training setup
optimizer = torch.optim.AdamW([
    {'params': model.module.segformer.parameters(), 'lr': 5e-6},
    {'params': model.module.decode_head.parameters(), 'lr': 5e-5}
])

num_epochs = 30
steps_per_epoch = len(train_dataloader)
total_steps = num_epochs * steps_per_epoch

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=[5e-5, 5e-4],
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    pct_start=0.3,
    anneal_strategy='cos'
)

scaler = GradScaler()
gradient_accumulation_steps = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training loop
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, 
                           scaler, device, gradient_accumulation_steps)
    val_loss = validate(model, val_dataloader, device)
    
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_loss': val_loss,
            'train_loss': train_loss
        }, 'best_model.pt')
        print(f"Saved new best model with validation loss: {val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs")
            break

print("Training completed!")

CUDA available: True
Number of GPUs: 2
GPU 0: Tesla T4
GPU 1: Tesla T4
Starting data preparation...
Initializing model and processor...


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

...

Model Parameters: 84,594,882
Using 2 GPUs!

Epoch 1/30


/tmp/ipykernel_31/2693918669.py:176: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/tmp/ipykernel_31/2693918669.py:58: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was ask

Batch: 0, Loss: 0.7051
Batch: 5, Loss: 0.7081
Batch: 10, Loss: 0.7014
Batch: 15, Loss: 0.6943
Batch: 20, Loss: 0.6492
Batch: 25, Loss: 0.6818
Batch: 30, Loss: 0.6723
Batch: 35, Loss: 0.6643
Batch: 40, Loss: 0.6563
Batch: 45, Loss: 0.6894


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1: Train Loss = 0.6776, Val Loss = 0.6120
Saved new best model with validation loss: 0.6120

Epoch 2/30
Batch: 0, Loss: 0.6412
Batch: 5, Loss: 0.6391
Batch: 10, Loss: 0.6498
Batch: 15, Loss: 0.6234
Batch: 20, Loss: 0.5952
Batch: 25, Loss: 0.6085
Batch: 30, Loss: 0.6365
Batch: 35, Loss: 0.5877
Batch: 40, Loss: 0.6053
Batch: 45, Loss: 0.6046
Epoch 2: Train Loss = 0.6251, Val Loss = 0.5499
Saved new best model with validation loss: 0.5499

Epoch 3/30
Batch: 0, Loss: 0.5552
Batch: 5, Loss: 0.6012
Batch: 10, Loss: 0.6128
Batch: 15, Loss: 0.5749
Batch: 20, Loss: 0.5750
Batch: 25, Loss: 0.5832
Batch: 30, Loss: 0.5652
Batch: 35, Loss: 0.6221
Batch: 40, Loss: 0.6072
Batch: 45, Loss: 0.5603
Epoch 3: Train Loss = 0.6006, Val Loss = 0.5155
Saved new best model with validation loss: 0.5155

Epoch 4/30
Batch: 0, Loss: 0.5317
Batch: 5, Loss: 0.5897
Batch: 10, Loss: 0.5831
Batch: 15, Loss: 0.5833
Batch: 20, Loss: 0.5630
Batch: 25, Loss: 0.6070
Batch: 30, Loss: 0.5715
Batch: 35, Loss: 0.5676
Batc

In [6]:
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode

# Create resize transform
resize_transform = Resize(
    size=(48, 48),
    interpolation=InterpolationMode.BILINEAR,
    antialias=True
)

# Evaluate model
model.eval()
X_test_reshaped = X_test.reshape(-1, 48, 48, 1)
X_test_normalized = X_test_reshaped / 255.0

print("Making predictions on test set...")
predictions = np.zeros((len(X_test_normalized), 48, 48))

for i, image in enumerate(X_test_normalized):
    if i % 20 == 0:
        print(f"Predicting image {i}/{len(X_test_normalized)}")
    pred = predict(image, model, processor)
    
    # Resize prediction using torchvision if necessary
    if pred.shape != (48, 48):
        pred_tensor = torch.from_numpy(pred).unsqueeze(0)  # Add batch and channel dims
        pred_resized = resize_transform(pred_tensor).squeeze().numpy()
        predictions[i] = pred_resized
    else:
        predictions[i] = pred

print(f"Ground truth shape: {y_test.reshape(-1, 48, 48).shape}")
print(f"Predictions shape: {predictions.shape}")

# Calculate balanced accuracy
y_true_flat = y_test.reshape(-1, 48, 48).flatten()
y_pred_flat = predictions.flatten()
y_pred_flat = (y_pred_flat > 0.5).astype(int)

print(f"Flattened shapes - True: {y_true_flat.shape}, Pred: {y_pred_flat.shape}")
balanced_acc = balanced_accuracy_score(y_true_flat, y_pred_flat)
print(f"Final Balanced Accuracy: {balanced_acc:.4f}")

Making predictions on test set...
Predicting image 0/110


NameError: name 'predict' is not defined

In [8]:
import torch
import numpy as np
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode
from sklearn.metrics import balanced_accuracy_score

def predict(image, model, processor):
    """
    Make prediction for a single image
    
    Args:
        image: numpy array of shape (48, 48, 1)
        model: trained SegFormer model
        processor: SegFormer processor
    
    Returns:
        numpy array of shape (48, 48) with probability predictions
    """
    model.eval()
    with torch.no_grad():
        # Convert image to RGB
        image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
        
        # Process image
        inputs = processor(
            images=image_rgb,
            return_tensors="pt",
            do_rescale=False
        )
        
        # Move to GPU
        pixel_values = inputs['pixel_values'].cuda()
        
        # Get predictions
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        
        # Convert to probabilities and move to CPU
        probs = torch.softmax(logits, dim=1)
        pred = probs[0, 1].cpu().numpy()  # Take class 1 probability
        
        return pred

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b3")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=2,
    ignore_mismatched_sizes=True
).cuda()

# Load the best checkpoint
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])

# Create resize transform
resize_transform = Resize(
    size=(48, 48),
    interpolation=InterpolationMode.BILINEAR,
    antialias=True
)

# Evaluate model
model.eval()
X_test_reshaped = X_test.reshape(-1, 48, 48, 1)
X_test_normalized = X_test_reshaped / 255.0

print("Making predictions on test set...")
predictions = np.zeros((len(X_test_normalized), 48, 48))

for i, image in enumerate(X_test_normalized):
    if i % 20 == 0:
        print(f"Predicting image {i}/{len(X_test_normalized)}")
    pred = predict(image, model, processor)
    
    # Resize prediction using torchvision if necessary
    if pred.shape != (48, 48):
        pred_tensor = torch.from_numpy(pred).unsqueeze(0)  # Add batch and channel dims
        pred_resized = resize_transform(pred_tensor).squeeze().numpy()
        predictions[i] = pred_resized
    else:
        predictions[i] = pred

print(f"Ground truth shape: {y_test.reshape(-1, 48, 48).shape}")
print(f"Predictions shape: {predictions.shape}")

# Calculate balanced accuracy
y_true_flat = y_test.reshape(-1, 48, 48).flatten()
y_pred_flat = predictions.flatten()
y_pred_flat = (y_pred_flat > 0.5).astype(int)

print(f"Flattened shapes - True: {y_true_flat.shape}, Pred: {y_pred_flat.shape}")
balanced_acc = balanced_accuracy_score(y_true_flat, y_pred_flat)
print(f"Final Balanced Accuracy: {balanced_acc:.4f}")

# Calculate additional metrics
tp = np.sum((y_true_flat == 1) & (y_pred_flat == 1))
tn = np.sum((y_true_flat == 0) & (y_pred_flat == 0))
fp = np.sum((y_true_flat == 0) & (y_pred_flat == 1))
fn = np.sum((y_true_flat == 1) & (y_pred_flat == 0))

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\nDetailed Metrics:")
print(f"Sensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_29/3824458066.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modu

RuntimeError: Error(s) in loading state_dict for SegformerForSemanticSegmentation:
	Missing key(s) in state_dict: "segformer.encoder.patch_embeddings.0.proj.weight", "segformer.encoder.patch_embeddings.0.proj.bias", "segformer.encoder.patch_embeddings.0.layer_norm.weight", "segformer.encoder.patch_embeddings.0.layer_norm.bias", "segformer.encoder.patch_embeddings.1.proj.weight", "segformer.encoder.patch_embeddings.1.proj.bias", "segformer.encoder.patch_embeddings.1.layer_norm.weight", "segformer.encoder.patch_embeddings.1.layer_norm.bias", "segformer.encoder.patch_embeddings.2.proj.weight", "segformer.encoder.patch_embeddings.2.proj.bias", "segformer.encoder.patch_embeddings.2.layer_norm.weight", "segformer.encoder.patch_embeddings.2.layer_norm.bias", "segformer.encoder.patch_embeddings.3.proj.weight", "segformer.encoder.patch_embeddings.3.proj.bias", "segformer.encoder.patch_embeddings.3.layer_norm.weight", "segformer.encoder.patch_embeddings.3.layer_norm.bias", "segformer.encoder.blo

In [9]:
import torch
import numpy as np
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode
from sklearn.metrics import balanced_accuracy_score

def remove_module_prefix(state_dict):
    """Remove 'module.' prefix from state_dict keys if present"""
    new_state_dict = {}
    for key, value in state_dict.items():
        if key.startswith('module.'):
            new_key = key[7:]  # Remove 'module.' prefix
        else:
            new_key = key
        new_state_dict[new_key] = value
    return new_state_dict

def predict(image, model, processor):
    model.eval()
    with torch.no_grad():
        image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
        inputs = processor(
            images=image_rgb,
            return_tensors="pt",
            do_rescale=False
        )
        pixel_values = inputs['pixel_values'].cuda()
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        pred = probs[0, 1].cpu().numpy()
        return pred

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b3")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=2,
    ignore_mismatched_sizes=True
).cuda()

# Load the best checkpoint
checkpoint = torch.load('best_model.pt')
state_dict = remove_module_prefix(checkpoint['model_state_dict'])
model.load_state_dict(state_dict)

# Create resize transform
resize_transform = Resize(
    size=(48, 48),
    interpolation=InterpolationMode.BILINEAR,
    antialias=True
)

# Evaluate model
model.eval()
X_test_reshaped = X_test.reshape(-1, 48, 48, 1)
X_test_normalized = X_test_reshaped / 255.0

print("Making predictions on test set...")
predictions = np.zeros((len(X_test_normalized), 48, 48))

for i, image in enumerate(X_test_normalized):
    if i % 20 == 0:
        print(f"Predicting image {i}/{len(X_test_normalized)}")
    pred = predict(image, model, processor)
    
    if pred.shape != (48, 48):
        pred_tensor = torch.from_numpy(pred).unsqueeze(0)
        pred_resized = resize_transform(pred_tensor).squeeze().numpy()
        predictions[i] = pred_resized
    else:
        predictions[i] = pred

print(f"Ground truth shape: {y_test.reshape(-1, 48, 48).shape}")
print(f"Predictions shape: {predictions.shape}")

# Calculate metrics
y_true_flat = y_test.reshape(-1, 48, 48).flatten()
y_pred_flat = predictions.flatten()
y_pred_flat = (y_pred_flat > 0.5).astype(int)

print(f"Flattened shapes - True: {y_true_flat.shape}, Pred: {y_pred_flat.shape}")
balanced_acc = balanced_accuracy_score(y_true_flat, y_pred_flat)
print(f"Final Balanced Accuracy: {balanced_acc:.4f}")

# Additional metrics
tp = np.sum((y_true_flat == 1) & (y_pred_flat == 1))
tn = np.sum((y_true_flat == 0) & (y_pred_flat == 0))
fp = np.sum((y_true_flat == 0) & (y_pred_flat == 1))
fn = np.sum((y_true_flat == 1) & (y_pred_flat == 0))

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\nDetailed Metrics:")
print(f"Sensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_29/2240498336.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modu

Making predictions on test set...
Predicting image 0/110
Predicting image 20/110
Predicting image 40/110
Predicting image 60/110
Predicting image 80/110
Predicting image 100/110
Ground truth shape: (110, 48, 48)
Predictions shape: (110, 48, 48)
Flattened shapes - True: (253440,), Pred: (253440,)
Final Balanced Accuracy: 0.8258

Detailed Metrics:
Sensitivity (True Positive Rate): 0.7271
Specificity (True Negative Rate): 0.9246
True Positives: 54176
True Negatives: 165435
False Positives: 13495
False Negatives: 20334


In [10]:
X_final = np.load('/kaggle/input/crater-segmentation/Xtest2_b.npy')

In [11]:
import torch
import numpy as np
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode

def remove_module_prefix(state_dict):
    """Remove 'module.' prefix from state_dict keys if present"""
    new_state_dict = {}
    for key, value in state_dict.items():
        if key.startswith('module.'):
            new_key = key[7:]  # Remove 'module.' prefix
        else:
            new_key = key
        new_state_dict[new_key] = value
    return new_state_dict

def predict(image, model, processor):
    model.eval()
    with torch.no_grad():
        image_rgb = np.repeat(image.reshape(48, 48, 1), 3, axis=-1)
        inputs = processor(
            images=image_rgb,
            return_tensors="pt",
            do_rescale=False
        )
        pixel_values = inputs['pixel_values'].cuda()
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        pred = probs[0, 1].cpu().numpy()
        return pred

# Load and prepare X_final
X_final = np.load('/kaggle/input/crater-segmentation/Xtest2_b.npy')
X_final_normalized = X_final / 255.0

# Initialize model and processor
processor = SegformerImageProcessor.from_pretrained("nvidia/mit-b3")
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=2,
    ignore_mismatched_sizes=True
).cuda()

# Load the best checkpoint
checkpoint = torch.load('best_model.pt')
state_dict = remove_module_prefix(checkpoint['model_state_dict'])
model.load_state_dict(state_dict)

# Create resize transform
resize_transform = Resize(
    size=(48, 48),
    interpolation=InterpolationMode.BILINEAR,
    antialias=True
)

# Make predictions
print("Making predictions on X_final...")
predictions = np.zeros((len(X_final_normalized), 48, 48))

for i, image in enumerate(X_final_normalized):
    if i % 20 == 0:
        print(f"Predicting image {i}/{len(X_final_normalized)}")
    pred = predict(image, model, processor)
    
    if pred.shape != (48, 48):
        pred_tensor = torch.from_numpy(pred).unsqueeze(0)
        pred_resized = resize_transform(pred_tensor).squeeze().numpy()
        predictions[i] = pred_resized
    else:
        predictions[i] = pred

# Convert predictions to binary (0 or 1)
predictions_binary = (predictions > 0.5).astype(np.uint8)

# Save predictions
#np.save('predictions_final.npy', predictions_binary)
print("Predictions saved to predictions_final.npy")

# Print some statistics
print("\nPrediction Statistics:")
print(f"Shape: {predictions_binary.shape}")
print(f"Number of positive predictions: {np.sum(predictions_binary == 1)}")
print(f"Number of negative predictions: {np.sum(predictions_binary == 0)}")
print(f"Percentage of positive predictions: {100 * np.mean(predictions_binary):.2f}%")

/opt/conda/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN thi

Making predictions on X_final...
Predicting image 0/196
Predicting image 20/196
Predicting image 40/196
Predicting image 60/196
Predicting image 80/196
Predicting image 100/196
Predicting image 120/196
Predicting image 140/196
Predicting image 160/196
Predicting image 180/196
Predictions saved to predictions_final.npy

Prediction Statistics:
Shape: (196, 48, 48)
Number of positive predictions: 114312
Number of negative predictions: 337272
Percentage of positive predictions: 25.31%


In [14]:
predictions_binary = predictions_binary.reshape(196,48**2)

In [16]:
predictions_binary.shape

(196, 2304)

In [18]:
np.save('final_predictions.npy',predictions_binary) ##FINAL

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

Xtrain2_a = np.load('/kaggle/input/datasetmachinelearning4/Xtrain2_a.npy')
Ytrain2_a = np.load('/kaggle/input/datasetmachinelearning4/Ytrain2_a.npy')
Xtrain2_b = np.load('/kaggle/input/datasetmachinelearning4/Xtrain2_b.npy')
Ytrain2_b = np.load('/kaggle/input/datasetmachinelearning4/Ytrain2_b.npy')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

# Simple CNN model
model_a = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(7, 7, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

def unet_model(input_size=(48, 48, 1)):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    
    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge5)
    
    conv6 = Conv2D(1, (1, 1), activation='sigmoid')(conv5)
    
    return Model(inputs=inputs, outputs=conv6)

model_b = unet_model()
model_b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import numpy as np

# Reshape Xtrain2_a to have the shape (num_samples, 7, 7, 1)
Xtrain2_a = Xtrain2_a.reshape(-1, 7, 7, 1)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

# Simple CNN model for format a
model_a = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(7, 7, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the models
model_a.fit(Xtrain2_a, Ytrain2_a, epochs=10, batch_size=32)
model_b.fit(Xtrain2_b, Ytrain2_b, epochs=10, batch_size=32)

import numpy as np
from sklearn.metrics import balanced_accuracy_score

# Predict
preds_a = model_a.predict(Xtrain2_a).flatten()
preds_b = model_b.predict(Xtrain2_b).flatten()

# Round predictions to get binary outputs
preds_a_rounded = np.round(preds_a)
preds_b_rounded = np.round(preds_b)

# Compute balanced accuracy
balanced_acc_a = balanced_accuracy_score(Ytrain2_a.flatten(), preds_a_rounded)
balanced_acc_b = balanced_accuracy_score(Ytrain2_b.flatten(), preds_b_rounded)

print("Balanced Accuracy for Format A:", balanced_acc_a)
print("Balanced Accuracy for Format B:", balanced_acc_b)